#### 把标签或在原图上，观察原图
#cv2.rectangle需要coco格式的标签， 把yolo格式转成coco格式

#需要注意win和linux的路径格式不同

In [1]:
import numpy as np 
import pandas as pd
import cv2
import glob
import os
from PIL import Image, ImageDraw
import PIL 

In [2]:
def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

In [3]:
def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes 

In [4]:
img_path = './fruit/test/JPEGImages/'
label_path = './fruit/test_yolo/*.txt'
output_path = './fruit/paint_test/'
color = (0,0,255)
img_type = '.jpg'
cls_dict = {'0': 'shoot'}
# cls_dict = {'0':'knife', '1':'scissors',  '2':'lighter', '3':'USBFlashDisk', '4':'pressure', '5':'plasticBottleWithaNozzle', 
#                 '6':'seal', '7':'battery'}

In [5]:
imgs = glob.glob(img_path)
labels = glob.glob(label_path)
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [6]:
for i , label in enumerate(labels):
    with open(label, 'r') as f:
        bboxes =  f.readlines()
        
    label_name = label.split('\\')[1].split('.')[0] #分割符要根据win还是ubuntu变化
    src =  cv2.imread(img_path + label_name+ img_type)
    img_h, img_w = src.shape[:2]
    for b in bboxes: #遍历每个object
        print(label_name)
        #tmp = b.split('\n')[0].split(' ')
        tmp = b.strip().split()
        if len(tmp) == 5:
            cls, xmid, ymid, w, h = tmp
        elif len(tmp) == 6:
            cls, xmid, ymid, w, h, _ = tmp
        yolobox = np.array([float(xmid),float(ymid), float(w), float(h)]) #yolo转coco格式
        cocobox = yolo2coco(yolobox, img_h,img_w)
        xmin, ymin, w, h = cocobox
        cv2.putText(src, cls_dict[cls], (int(xmin), int(ymin)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.rectangle(src, (int(xmin), int(ymin)), (int(xmin + w), int(ymin + h)), color, 3)
    cv2.imwrite(output_path+label_name+'.jpg', src)

shoot108_2
shoot114_1
shoot114_1
shoot114_3
shoot114_3
shoot11_5
shoot11_5
shoot11_5
shoot121_2
shoot121_2
shoot121_2
shoot121_2
shoot125_2
shoot125_2
shoot125_2
shoot125_2
shoot125_2
shoot125_2
shoot140_2
shoot140_2
shoot141_2
shoot143_2
shoot147_3
shoot147_3
shoot147_3
shoot14_4
shoot14_4
shoot14_7
shoot151_2
shoot151_2
shoot151_2
shoot160_2
shoot160_2
shoot160_2
shoot160_3
shoot16_2
shoot16_2
shoot16_4
shoot16_4
shoot16_4
shoot16_4
shoot185_2
shoot185_2
shoot18_5
shoot18_5
shoot18_5
shoot198_2
shoot198_2
shoot20_5
shoot20_5
shoot20_5
shoot20_5
shoot20_7
shoot20_8
shoot210_2
shoot210_2
shoot214_3
shoot217_2
shoot217_2
shoot217_2
shoot217_2
shoot217_2
shoot217_2
shoot219_2
shoot226_5
shoot226_5
shoot226_5
shoot228_5
shoot228_5
shoot231_5
shoot231_5
shoot231_5
shoot232_6
shoot236_6
shoot236_6
shoot23_1
shoot23_1
shoot23_1
shoot23_1
shoot23_8
shoot242_4
shoot242_4
shoot242_4
shoot25_8
shoot266_8
shoot266_8
shoot275_2
shoot278_9
shoot278_9
shoot278_9
shoot283_1
shoot283_1
shoot283_1
shoo

###  基于voc格式

In [7]:
import cv2
import os 
import xml.etree.ElementTree as ET

In [11]:
def xml2txt(txt_path, image_path, path, labels):
    cnt = 0
    # 遍历图片文件夹
    for (root, dirname, files) in os.walk(image_path):
        print(root,dirname,files)
        # 获取图片名
        for ft in files:
            # ft是图片名字+扩展名，替换txt,xml格式
            ftxt = ft.replace(ft.split('.')[1], 'txt')
            fxml = ft.replace(ft.split('.')[1], 'xml')
            # xml文件路径
            xml_path = os.path.join(path, fxml)
            # txt文件路径
            ftxt_path = os.path.join(txt_path, ftxt)
            # 解析xml
            tree = ET.parse(xml_path)
            root = tree.getroot()
            # 获取weight,height
            size = root.find('size')
            w = size.find('width').text
            h = size.find('height').text
            dw = 1/int(w)
            dh = 1/int(h)
            # 初始化line
            line = ''
            for item in root.findall('object'):
                # 提取label,并获取索引
                label = item.find('name').text
                label = labels.index(label)
                # 提取信息labels, x, y, w, h 
                # 多框转化
                for box in item.findall('bndbox'):
                    xmin = float(box.find('xmin').text)
                    ymin = float(box.find('ymin').text)
                    xmax = float(box.find('xmax').text)
                    ymax = float(box.find('ymax').text)
                    print(xmin,ymin,xmax,ymax)

                    # x, y, w, h归一化
                    center_x = ((xmin + xmax) / 2) * dw
                    center_y = ((ymin + ymax) / 2) * dh
                    bbox_width = (xmax-xmin) * dw
                    bbox_height = (ymax-ymin) * dh
                    print(center_x,center_y,bbox_width,bbox_height)

                    # 传入信息，txt是字符串形式
                    line += '{} {} {} {} {}'.format(label,center_x,center_y,bbox_width,bbox_height) + '\n'              

            # 将txt信息写入文件
            with open(ftxt_path, 'w') as f_txt:
                f_txt.write(line)
            cnt += 1
            print('文件数量：', cnt)

In [13]:
txt_path = os.path.join('./fruit/train/labels/')  # yolo存放生成txt的文件目录
image_path = os.path.join('./fruit/train/JPEGImages/')  # 存放图片的文件目录
path = os.path.join('./fruit/train/Annotations/')  # 存放xml的文件目录
labels = ['shoot']  # 用于获取label位置
xml2txt(txt_path, image_path, path, labels)

./fruit/train/JPEGImages/ [] ['shoot0_5.jpg', 'shoot101_2.jpg', 'shoot10_5.jpg', 'shoot110_3.jpg', 'shoot127_1.jpg', 'shoot12_4.jpg', 'shoot134_2.jpg', 'shoot136_2.jpg', 'shoot13_5.jpg', 'shoot144_2.jpg', 'shoot148_1.jpg', 'shoot148_3.jpg', 'shoot14_5.jpg', 'shoot14_8.jpg', 'shoot150_2.jpg', 'shoot151_3.jpg', 'shoot152_3.jpg', 'shoot153_2.jpg', 'shoot156_2.jpg', 'shoot158_1.jpg', 'shoot164_3.jpg', 'shoot165_1.jpg', 'shoot166_3.jpg', 'shoot169_2.jpg', 'shoot16_3.jpg', 'shoot173_2.jpg', 'shoot18_4.jpg', 'shoot192_2.jpg', 'shoot195_2.jpg', 'shoot197_2.jpg', 'shoot198_1.jpg', 'shoot19_4.jpg', 'shoot19_5.jpg', 'shoot1_2.jpg', 'shoot212_2.jpg', 'shoot213_3.jpg', 'shoot215_2.jpg', 'shoot216_2.jpg', 'shoot218_2.jpg', 'shoot21_5.jpg', 'shoot220_2.jpg', 'shoot226_3.jpg', 'shoot229_5.jpg', 'shoot230_6.jpg', 'shoot241_5.jpg', 'shoot24_4.jpg', 'shoot251_2.jpg', 'shoot253_6.jpg', 'shoot254_5.jpg', 'shoot254_6.jpg', 'shoot257_5.jpg', 'shoot258_6.jpg', 'shoot25_9.jpg', 'shoot260_6.jpg', 'shoot263_5.jp

文件数量： 61
437.0 233.0 510.0 367.0
0.778782894736842 0.4934210526315789 0.12006578947368421 0.22039473684210525
387.0 301.0 441.0 381.0
0.6809210526315789 0.5608552631578947 0.08881578947368421 0.13157894736842105
489.0 420.0 534.0 500.0
0.841282894736842 0.756578947368421 0.07401315789473684 0.13157894736842105
527.0 443.0 563.0 517.0
0.8963815789473684 0.7894736842105263 0.05921052631578947 0.12171052631578946
558.0 467.0 608.0 553.0
0.9588815789473684 0.8388157894736842 0.08223684210526315 0.14144736842105263
405.0 32.0 466.0 90.0
0.716282894736842 0.10032894736842105 0.10032894736842105 0.09539473684210525
369.0 38.0 422.0 107.0
0.6504934210526315 0.11924342105263157 0.08717105263157894 0.11348684210526315
文件数量： 62
270.0 182.0 469.0 326.0
0.6077302631578947 0.41776315789473684 0.32730263157894735 0.23684210526315788
391.0 166.0 609.0 314.0
0.8223684210526315 0.39473684210526316 0.35855263157894735 0.24342105263157893
文件数量： 63
79.0 545.0 231.0 604.0
0.25493421052631576 0.9449013157894

文件数量： 117
114.0 127.0 237.0 390.0
0.2886513157894737 0.4251644736842105 0.20230263157894735 0.4325657894736842
178.0 250.0 341.0 456.0
0.42680921052631576 0.5805921052631579 0.26809210526315785 0.3388157894736842
文件数量： 118
11.0 383.0 206.0 542.0
0.17845394736842105 0.7606907894736842 0.32072368421052627 0.26151315789473684
157.0 322.0 285.0 500.0
0.36348684210526316 0.6759868421052632 0.21052631578947367 0.29276315789473684
446.0 414.0 609.0 598.0
0.8675986842105262 0.8322368421052632 0.26809210526315785 0.3026315789473684
文件数量： 119
44.0 165.0 388.0 364.0
0.35526315789473684 0.4350328947368421 0.5657894736842105 0.32730263157894735
文件数量： 120
21.0 419.0 190.0 609.0
0.17351973684210525 0.8453947368421052 0.27796052631578944 0.3125
21.0 326.0 158.0 499.0
0.14720394736842105 0.678453947368421 0.22532894736842105 0.2845394736842105
143.0 444.0 263.0 576.0
0.3338815789473684 0.8388157894736842 0.19736842105263158 0.21710526315789472
文件数量： 121
2.0 209.0 180.0 500.0
0.14967105263157893 0.58305

80.0 8.0 329.0 239.0
0.33634868421052627 0.203125 0.4095394736842105 0.37993421052631576
文件数量： 160
199.0 251.0 273.0 376.0
0.3881578947368421 0.515625 0.12171052631578946 0.20559210526315788
34.0 353.0 108.0 425.0
0.11677631578947367 0.6398026315789473 0.12171052631578946 0.11842105263157894
471.0 298.0 547.0 431.0
0.8371710526315789 0.5995065789473684 0.125 0.21875
315.0 446.0 365.0 522.0
0.5592105263157895 0.7960526315789473 0.08223684210526315 0.125
文件数量： 161
52.0 166.0 111.0 243.0
0.13404605263157893 0.33634868421052627 0.09703947368421052 0.12664473684210525
262.0 142.0 309.0 206.0
0.4695723684210526 0.28618421052631576 0.07730263157894736 0.10526315789473684
文件数量： 162
94.0 510.0 207.0 596.0
0.2475328947368421 0.9095394736842105 0.18585526315789472 0.14144736842105263
270.0 442.0 350.0 521.0
0.5098684210526315 0.7919407894736842 0.13157894736842105 0.1299342105263158
286.0 326.0 344.0 427.0
0.5180921052631579 0.6192434210526315 0.09539473684210525 0.16611842105263158
文件数量： 163
63.

0.26973684210526316 0.8215460526315789 0.12171052631578946 0.14309210526315788
文件数量： 220
342.0 159.0 456.0 282.0
0.65625 0.3626644736842105 0.1875 0.20230263157894735
167.0 136.0 274.0 257.0
0.3626644736842105 0.3231907894736842 0.17598684210526314 0.19901315789473684
文件数量： 221
364.0 251.0 485.0 388.0
0.6981907894736842 0.5254934210526315 0.19901315789473684 0.22532894736842105
441.0 344.0 515.0 465.0
0.7861842105263157 0.6652960526315789 0.12171052631578946 0.19901315789473684
534.0 359.0 593.0 452.0
0.9268092105263157 0.6669407894736842 0.09703947368421052 0.15296052631578946
文件数量： 222
237.0 311.0 374.0 454.0
0.5024671052631579 0.6291118421052632 0.22532894736842105 0.23519736842105263
337.0 277.0 474.0 429.0
0.6669407894736842 0.5805921052631579 0.22532894736842105 0.25
173.0 115.0 389.0 282.0
0.4621710526315789 0.32648026315789475 0.35526315789473684 0.2746710526315789
文件数量： 223
466.0 104.0 609.0 254.0
0.8840460526315789 0.2944078947368421 0.23519736842105263 0.24671052631578946
文件

文件数量： 289
302.0 321.0 382.0 460.0
0.5625 0.6422697368421052 0.13157894736842105 0.22861842105263155
348.0 374.0 430.0 467.0
0.6398026315789473 0.6916118421052632 0.13486842105263158 0.15296052631578946
263.0 339.0 316.0 461.0
0.4761513157894737 0.6578947368421052 0.08717105263157894 0.2006578947368421
文件数量： 290
95.0 8.0 178.0 133.0
0.22450657894736842 0.11595394736842105 0.13651315789473684 0.20559210526315788
文件数量： 291
153.0 446.0 286.0 604.0
0.36101973684210525 0.8634868421052632 0.21875 0.2598684210526316
400.0 515.0 533.0 609.0
0.7672697368421052 0.9243421052631579 0.21875 0.15460526315789472
182.0 352.0 280.0 462.0
0.37993421052631576 0.669407894736842 0.1611842105263158 0.18092105263157893
286.0 294.0 383.0 467.0
0.5501644736842105 0.6258223684210525 0.1595394736842105 0.2845394736842105
文件数量： 292
48.0 168.0 181.0 299.0
0.18832236842105263 0.3840460526315789 0.21875 0.21546052631578946
465.0 469.0 585.0 604.0
0.8634868421052632 0.8824013157894737 0.19736842105263158 0.22203947368

23.0 63.0 214.0 248.0
0.19490131578947367 0.2557565789473684 0.31414473684210525 0.3042763157894737
69.0 248.0 278.0 379.0
0.28536184210526316 0.515625 0.34375 0.21546052631578946
204.0 58.0 449.0 233.0
0.5370065789473684 0.23930921052631576 0.40296052631578944 0.287828947368421
文件数量： 333
140.0 477.0 387.0 609.0
0.43338815789473684 0.8930921052631579 0.40625 0.21710526315789472
71.0 2.0 263.0 209.0
0.2746710526315789 0.17351973684210525 0.3157894736842105 0.34046052631578944
文件数量： 334
67.0 349.0 207.0 519.0
0.22532894736842105 0.7138157894736842 0.23026315789473684 0.27960526315789475
234.0 190.0 417.0 428.0
0.5353618421052632 0.5082236842105263 0.30098684210526316 0.3914473684210526
364.0 306.0 538.0 415.0
0.7417763157894737 0.5929276315789473 0.28618421052631576 0.17927631578947367
文件数量： 335
101.0 314.0 247.0 515.0
0.28618421052631576 0.6817434210526315 0.24013157894736842 0.33059210526315785
257.0 253.0 427.0 477.0
0.5625 0.600328947368421 0.27960526315789475 0.3684210526315789
171.

文件数量： 373
547.0 278.0 609.0 390.0
0.950657894736842 0.5493421052631579 0.10197368421052631 0.18421052631578946
451.0 141.0 505.0 219.0
0.7861842105263157 0.29605263157894735 0.08881578947368421 0.1282894736842105
218.0 240.0 348.0 337.0
0.46546052631578944 0.4745065789473684 0.2138157894736842 0.1595394736842105
2.0 134.0 89.0 248.0
0.07483552631578946 0.31414473684210525 0.14309210526315788 0.1875
381.0 396.0 584.0 545.0
0.7935855263157894 0.7738486842105263 0.3338815789473684 0.2450657894736842
文件数量： 374
440.0 81.0 602.0 339.0
0.856907894736842 0.34539473684210525 0.2664473684210526 0.42434210526315785
文件数量： 375
48.0 199.0 195.0 386.0
0.19983552631578946 0.48108552631578944 0.24177631578947367 0.3075657894736842
132.0 428.0 254.0 581.0
0.31743421052631576 0.8297697368421052 0.2006578947368421 0.25164473684210525
287.0 344.0 496.0 523.0
0.6439144736842105 0.7129934210526315 0.34375 0.2944078947368421
文件数量： 376
30.0 39.0 139.0 154.0
0.13898026315789472 0.15871710526315788 0.17927631578

0.5822368421052632 0.35855263157894735 0.10197368421052631 0.15789473684210525
222.0 279.0 278.0 364.0
0.41118421052631576 0.528782894736842 0.09210526315789473 0.13980263157894737
文件数量： 413
258.0 473.0 340.0 597.0
0.4917763157894737 0.8799342105263157 0.13486842105263158 0.20394736842105263
141.0 326.0 177.0 394.0
0.26151315789473684 0.5921052631578947 0.05921052631578947 0.11184210526315788
176.0 331.0 207.0 393.0
0.31496710526315785 0.5953947368421052 0.05098684210526316 0.10197368421052631
325.0 520.0 380.0 578.0
0.5797697368421052 0.9029605263157894 0.09046052631578946 0.09539473684210525
416.0 368.0 447.0 410.0
0.709703947368421 0.6398026315789473 0.05098684210526316 0.06907894736842105
文件数量： 414
199.0 378.0 278.0 487.0
0.39226973684210525 0.7113486842105263 0.1299342105263158 0.17927631578947367
464.0 219.0 515.0 282.0
0.8050986842105262 0.4120065789473684 0.08388157894736842 0.10361842105263157
255.0 79.0 312.0 138.0
0.4662828947368421 0.17845394736842105 0.09375 0.097039473684

In [6]:
######改名######
import os
path= './xray/train/label3/'  

#获取该目录下所有文件，存入列表中
f=os.listdir(path)

n=0
for i in f:

    #设置旧文件名（就是路径+文件名）
    oldname=path+f[n]
    number = oldname.split('/')[-1].split('.')[0]

    #设置新文件名
    newname= './xray/train/labels/' + str((int(number) + 2706)).zfill(6) + '.txt' 
    
    #用os模块中的rename方法对文件改名
    os.rename(oldname,newname)
    print(oldname,'======>',newname)

    n+=1


./xray/train/label3/000001.txt ======> ./xray/train/labels/002707.txt
./xray/train/label3/000002.txt ======> ./xray/train/labels/002708.txt
./xray/train/label3/000003.txt ======> ./xray/train/labels/002709.txt
./xray/train/label3/000004.txt ======> ./xray/train/labels/002710.txt
./xray/train/label3/000005.txt ======> ./xray/train/labels/002711.txt
./xray/train/label3/000006.txt ======> ./xray/train/labels/002712.txt
./xray/train/label3/000007.txt ======> ./xray/train/labels/002713.txt
./xray/train/label3/000008.txt ======> ./xray/train/labels/002714.txt
./xray/train/label3/000009.txt ======> ./xray/train/labels/002715.txt
./xray/train/label3/000010.txt ======> ./xray/train/labels/002716.txt
./xray/train/label3/000011.txt ======> ./xray/train/labels/002717.txt
./xray/train/label3/000012.txt ======> ./xray/train/labels/002718.txt
./xray/train/label3/000013.txt ======> ./xray/train/labels/002719.txt
./xray/train/label3/000014.txt ======> ./xray/train/labels/002720.txt
./xray/train/label3/

./xray/train/label3/000126.txt ======> ./xray/train/labels/002832.txt
./xray/train/label3/000127.txt ======> ./xray/train/labels/002833.txt
./xray/train/label3/000128.txt ======> ./xray/train/labels/002834.txt
./xray/train/label3/000129.txt ======> ./xray/train/labels/002835.txt
./xray/train/label3/000130.txt ======> ./xray/train/labels/002836.txt
./xray/train/label3/000131.txt ======> ./xray/train/labels/002837.txt
./xray/train/label3/000132.txt ======> ./xray/train/labels/002838.txt
./xray/train/label3/000133.txt ======> ./xray/train/labels/002839.txt
./xray/train/label3/000134.txt ======> ./xray/train/labels/002840.txt
./xray/train/label3/000135.txt ======> ./xray/train/labels/002841.txt
./xray/train/label3/000136.txt ======> ./xray/train/labels/002842.txt
./xray/train/label3/000137.txt ======> ./xray/train/labels/002843.txt
./xray/train/label3/000138.txt ======> ./xray/train/labels/002844.txt
./xray/train/label3/000139.txt ======> ./xray/train/labels/002845.txt
./xray/train/label3/

./xray/train/label3/000255.txt ======> ./xray/train/labels/002961.txt
./xray/train/label3/000256.txt ======> ./xray/train/labels/002962.txt
./xray/train/label3/000257.txt ======> ./xray/train/labels/002963.txt
./xray/train/label3/000258.txt ======> ./xray/train/labels/002964.txt
./xray/train/label3/000259.txt ======> ./xray/train/labels/002965.txt
./xray/train/label3/000260.txt ======> ./xray/train/labels/002966.txt
./xray/train/label3/000261.txt ======> ./xray/train/labels/002967.txt
./xray/train/label3/000262.txt ======> ./xray/train/labels/002968.txt
./xray/train/label3/000263.txt ======> ./xray/train/labels/002969.txt
./xray/train/label3/000264.txt ======> ./xray/train/labels/002970.txt
./xray/train/label3/000265.txt ======> ./xray/train/labels/002971.txt
./xray/train/label3/000266.txt ======> ./xray/train/labels/002972.txt
./xray/train/label3/000267.txt ======> ./xray/train/labels/002973.txt
./xray/train/label3/000268.txt ======> ./xray/train/labels/002974.txt
./xray/train/label3/

./xray/train/label3/000493.txt ======> ./xray/train/labels/003199.txt
./xray/train/label3/000494.txt ======> ./xray/train/labels/003200.txt
./xray/train/label3/000495.txt ======> ./xray/train/labels/003201.txt
./xray/train/label3/000496.txt ======> ./xray/train/labels/003202.txt
./xray/train/label3/000497.txt ======> ./xray/train/labels/003203.txt
./xray/train/label3/000498.txt ======> ./xray/train/labels/003204.txt
./xray/train/label3/000499.txt ======> ./xray/train/labels/003205.txt
./xray/train/label3/000500.txt ======> ./xray/train/labels/003206.txt
./xray/train/label3/000501.txt ======> ./xray/train/labels/003207.txt
./xray/train/label3/000502.txt ======> ./xray/train/labels/003208.txt
./xray/train/label3/000503.txt ======> ./xray/train/labels/003209.txt
./xray/train/label3/000504.txt ======> ./xray/train/labels/003210.txt
./xray/train/label3/000505.txt ======> ./xray/train/labels/003211.txt
./xray/train/label3/000506.txt ======> ./xray/train/labels/003212.txt
./xray/train/label3/

./xray/train/label3/000611.txt ======> ./xray/train/labels/003317.txt
./xray/train/label3/000612.txt ======> ./xray/train/labels/003318.txt
./xray/train/label3/000613.txt ======> ./xray/train/labels/003319.txt
./xray/train/label3/000614.txt ======> ./xray/train/labels/003320.txt
./xray/train/label3/000615.txt ======> ./xray/train/labels/003321.txt
./xray/train/label3/000616.txt ======> ./xray/train/labels/003322.txt
./xray/train/label3/000617.txt ======> ./xray/train/labels/003323.txt
./xray/train/label3/000618.txt ======> ./xray/train/labels/003324.txt
./xray/train/label3/000619.txt ======> ./xray/train/labels/003325.txt
./xray/train/label3/000620.txt ======> ./xray/train/labels/003326.txt
./xray/train/label3/000621.txt ======> ./xray/train/labels/003327.txt
./xray/train/label3/000622.txt ======> ./xray/train/labels/003328.txt
./xray/train/label3/000623.txt ======> ./xray/train/labels/003329.txt
./xray/train/label3/000624.txt ======> ./xray/train/labels/003330.txt
./xray/train/label3/

./xray/train/label3/000747.txt ======> ./xray/train/labels/003453.txt
./xray/train/label3/000748.txt ======> ./xray/train/labels/003454.txt
./xray/train/label3/000749.txt ======> ./xray/train/labels/003455.txt
./xray/train/label3/000750.txt ======> ./xray/train/labels/003456.txt
./xray/train/label3/000751.txt ======> ./xray/train/labels/003457.txt
./xray/train/label3/000752.txt ======> ./xray/train/labels/003458.txt
./xray/train/label3/000753.txt ======> ./xray/train/labels/003459.txt
./xray/train/label3/000754.txt ======> ./xray/train/labels/003460.txt
./xray/train/label3/000755.txt ======> ./xray/train/labels/003461.txt
./xray/train/label3/000756.txt ======> ./xray/train/labels/003462.txt
./xray/train/label3/000757.txt ======> ./xray/train/labels/003463.txt
./xray/train/label3/000758.txt ======> ./xray/train/labels/003464.txt
./xray/train/label3/000759.txt ======> ./xray/train/labels/003465.txt
./xray/train/label3/000760.txt ======> ./xray/train/labels/003466.txt
./xray/train/label3/

./xray/train/label3/000991.txt ======> ./xray/train/labels/003697.txt
./xray/train/label3/000992.txt ======> ./xray/train/labels/003698.txt
./xray/train/label3/000993.txt ======> ./xray/train/labels/003699.txt
./xray/train/label3/000994.txt ======> ./xray/train/labels/003700.txt
./xray/train/label3/000995.txt ======> ./xray/train/labels/003701.txt
./xray/train/label3/000996.txt ======> ./xray/train/labels/003702.txt
./xray/train/label3/000997.txt ======> ./xray/train/labels/003703.txt
./xray/train/label3/000998.txt ======> ./xray/train/labels/003704.txt
./xray/train/label3/000999.txt ======> ./xray/train/labels/003705.txt
./xray/train/label3/001000.txt ======> ./xray/train/labels/003706.txt
./xray/train/label3/001001.txt ======> ./xray/train/labels/003707.txt
./xray/train/label3/001002.txt ======> ./xray/train/labels/003708.txt
./xray/train/label3/001003.txt ======> ./xray/train/labels/003709.txt
./xray/train/label3/001004.txt ======> ./xray/train/labels/003710.txt
./xray/train/label3/

./xray/train/label3/001118.txt ======> ./xray/train/labels/003824.txt
./xray/train/label3/001119.txt ======> ./xray/train/labels/003825.txt
./xray/train/label3/001120.txt ======> ./xray/train/labels/003826.txt
./xray/train/label3/001121.txt ======> ./xray/train/labels/003827.txt
./xray/train/label3/001122.txt ======> ./xray/train/labels/003828.txt
./xray/train/label3/001123.txt ======> ./xray/train/labels/003829.txt
./xray/train/label3/001124.txt ======> ./xray/train/labels/003830.txt
./xray/train/label3/001125.txt ======> ./xray/train/labels/003831.txt
./xray/train/label3/001126.txt ======> ./xray/train/labels/003832.txt
./xray/train/label3/001127.txt ======> ./xray/train/labels/003833.txt
./xray/train/label3/001128.txt ======> ./xray/train/labels/003834.txt
./xray/train/label3/001129.txt ======> ./xray/train/labels/003835.txt
./xray/train/label3/001130.txt ======> ./xray/train/labels/003836.txt
./xray/train/label3/001131.txt ======> ./xray/train/labels/003837.txt
./xray/train/label3/

./xray/train/label3/001254.txt ======> ./xray/train/labels/003960.txt
./xray/train/label3/001255.txt ======> ./xray/train/labels/003961.txt
./xray/train/label3/001256.txt ======> ./xray/train/labels/003962.txt
./xray/train/label3/001257.txt ======> ./xray/train/labels/003963.txt
./xray/train/label3/001258.txt ======> ./xray/train/labels/003964.txt
./xray/train/label3/001259.txt ======> ./xray/train/labels/003965.txt
./xray/train/label3/001260.txt ======> ./xray/train/labels/003966.txt
./xray/train/label3/001261.txt ======> ./xray/train/labels/003967.txt
./xray/train/label3/001262.txt ======> ./xray/train/labels/003968.txt
./xray/train/label3/001263.txt ======> ./xray/train/labels/003969.txt
./xray/train/label3/001264.txt ======> ./xray/train/labels/003970.txt
./xray/train/label3/001265.txt ======> ./xray/train/labels/003971.txt
./xray/train/label3/001266.txt ======> ./xray/train/labels/003972.txt
./xray/train/label3/001267.txt ======> ./xray/train/labels/003973.txt
./xray/train/label3/